<a href="https://colab.research.google.com/github/arputtick/Bird-is-the-Word/blob/main/src/Bird_is_the_Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bird is the Word


# Preamble/Setup

In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [2]:
pip install transformers -qq

     |████████████████████████████████| 5.8 MB 4.5 MB/s 
     |████████████████████████████████| 7.6 MB 51.8 MB/s 
     |████████████████████████████████| 182 kB 97.7 MB/s 


In [3]:
pip install geopandas -qq

     |████████████████████████████████| 1.1 MB 4.5 MB/s 
     |████████████████████████████████| 7.8 MB 92.0 MB/s 
     |████████████████████████████████| 16.6 MB 85.9 MB/s 


In [4]:
pip install meteostat -qq

In [5]:
pip install wikipedia-api -qq

In [6]:
pip install openai -qq

     |████████████████████████████████| 44 kB 2.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 146 kB 8.4 MB/s 


In [7]:
import numpy as np
import tensorflow as tf
import torch
import pandas as pd
import transformers
import requests, json
import os
import matplotlib.pyplot as plt
import nltk
nltk.download('punkt')

import geopandas as gpd
import geopy
import wikipediaapi

from geopy import Point as point
from shapely.geometry import MultiPoint
from geopy.distance import distance
from geopy.distance import geodesic

from meteostat import Hourly
from datetime import datetime
from meteostat import Point, Daily
from meteostat.units import condition

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [8]:
from geopy.geocoders import Nominatim
  
# initialize Nominatim API
geolocator = Nominatim(user_agent="Waldrapp")

In [9]:
# from google.colab import drive
# drive.mount('/content/drive')

# OpenAI API Access Settings

In [10]:
import os
import openai
openai.organization = "<YOUR ORGANIZATION ID HERE>"
openai.api_key = "<YOUR KEY HERE>"

# Set Personality Parameters

In [66]:
## Select Bird and Timeframe

# individual = "Marcello"
individual = "Laura"
# individual = "Enea"
# individual = "Jazu"


In [67]:
## Set intro and model personality

if individual == 'Marcello':
    context = "Greetings and praise Allah. I am Marcello, and male I am a northern bald ibis. I am an endangered bird. I am an ancient muslim traveler."
    style = "biblical style"
    personality = 'muslim_travel'

if individual == 'Enea':
    context = "Kamusta. I am Enea, and I am a bird. I am a female northern bald ibis. I am also a filipino shaman who speaks for the anito spirits of wind, fire, water and earth."
    style = "first person"
    personality = 'filipino'

if individual == 'Laura':
    context = "Hello, my name is Laura, and I am a queer non-binary northern bald ibis."
    style = ""
    personality = 'eco-feminist'

# Data, model and story functions

## Load Bird Data

In [73]:
# s_path = os.getcwd()
# print(s_path) 
# data_path = s_path + '/drive/MyDrive/Bird is the word/data/gps/'

url = "https://raw.githubusercontent.com/arputtick/Bird-is-the-Word/main/data/beacon/" + individual + ".csv"

# df = pd.read_csv(data_path+ individual +'.csv') # Laura.csv Marcello.csv Enea.csv
df = pd.read_csv(url)

# convert to pd datetime and set UTC_datetime as index
df = df.set_index('UTC_datetime')
df.index = pd.to_datetime(df.index) # convert to datetime object
df = df[df["Latitude"].apply(lambda x: x != 0)] #filter data without gps fix
df

,device_id,UTC_date,UTC_time,datatype,satcount,U_bat_mV,bat_soc_pct,solar_I_mA,hdop,Latitude,...,speed_km_h,direction_deg,temperature_C,mag_x,mag_y,mag_z,acc_x,acc_y,acc_z,Unnamed: 22
UTC_datetime,,,,,,,,,,,,,,,,,,,,,
2021-01-05 13:28:36,181742,2021-01-05,13:28:36,GPS,5,3608,0,12,1.2,42.476833,...,3,36,17,-19,384,35,-347,-642,951,NaN
2021-01-06 11:13:34,181742,2021-01-06,11:13:34,GPS,5,3611,0,13,1.9,42.481167,...,46,68,15,-99,254,48,132,-508,811,NaN
2021-01-20 14:53:06,181742,2021-01-20,14:53:06,GPS,5,3678,1,10,1.4,42.476883,...,1,119,17,-210,366,-44,328,-703,702,NaN
2021-01-20 14:53:06,181742,2021-01-20,14:53:06,GPS,5,3678,1,10,1.4,42.476883,...,1,119,17,-210,366,-44,328,-703,702,NaN
2021-01-26 10:42:36,181742,2021-01-26,10:42:36,GPS,5,3611,0,17,1.1,42.476700,...,0,315,24,24,54,15,340,-584,796,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-18 06:45:22,181742,2022-01-18,06:45:22,GPS,6,3774,28,0,2.7,42.495319,...,48,315,6,123,13,5,156,-440,202,NaN
2022-01-18 18:44:31,181742,2022-01-18,18:44:31,GPS,7,3785,31,0,1.5,42.482155,...,0,2,11,-187,362,38,292,-531,853,NaN
2022-01-19 06:44:20,181742,2022-01-19,06:44:20,GPS,7,3768,27,0,2.0,42.508194,...,29,2,6,80,233,92,203,159,436,NaN


## Load Trained Language Model and Generate 

In [68]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [69]:
from transformers import AutoModelForCausalLM
# model = AutoModelForCausalLM.from_pretrained('/content/drive/MyDrive/Bird is the word/models/GPT-Neo_' + personality)
model = AutoModelForCausalLM.from_pretrained('arputtick/GPT_Neo_' + personality)
model.to(device)

Downloading:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/5.36G [00:00<?, ?B/s]

GPTNeoForCausalLM(
  (transformer): GPTNeoModel(
    (wte): Embedding(50257, 2048)
    (wpe): Embedding(2048, 2048)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTNeoBlock(
        (ln_1): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (attn): GPTNeoAttention(
          (attention): GPTNeoSelfAttention(
            (attn_dropout): Dropout(p=0.0, inplace=False)
            (resid_dropout): Dropout(p=0.0, inplace=False)
            (k_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (v_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
            (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
          )
        )
        (ln_2): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
        (mlp): GPTNeoMLP(
          (c_fc): Linear(in_features=2048, out_features=8192, bias=True)
          (c_proj): Linear(

In [70]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neo-1.3B")

In [71]:
def generate_sentences(text, length):
    prompt = tokenizer(text, return_tensors='pt')
    prompt = {key: value.to(device) for key, value in prompt.items()}
    out = model.generate(**prompt, min_length=length, max_length=length, do_sample=True)
    generated = tokenizer.decode(out[0])
    sentences = nltk.sent_tokenize(generated)

    if sentences[-1][-1] == ".":
        output = sentences

    else:
        output = sentences[:-1]

    return " ".join(output)

In [72]:
print(generate_sentences('My name is Enea. I am a filipina northern bald ibis. I am in a latent space',100))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


My name is Enea. I am a filipina northern bald ibis. I am in a latent space with humans, connected to them, a part of them. I am not a human, a biological, artificial, or social presence. I am a mind in its own right, a field in its own right.


## Get Location, Weather and Distance Traveled

### Get weather from datetime

In [19]:
def get_weather_conditions(datetime):
    lat = df.loc[datetime]['Latitude']
    lon = df.loc[datetime]['Longitude']
    
    weather = Hourly(Point(lat,lon), datetime, datetime.ceil('H'))
    weather = weather.fetch()
    
    temp = weather['temp'][0]
    coco = weather['coco'][0]
    
    return temp, coco

### Distance per day

In [20]:
# Compute Distance per day

df['point'] = df.apply(lambda row: point(latitude=row['Latitude'], longitude=row['Longitude']), axis=1)
df['point_next'] = df['point'].shift(1)
df.loc[df['point_next'].isna(), 'point_next'] = None
df['distance_km'] = df.apply(lambda row: distance(row['point'], row['point_next']).km if row['point_next'] is not None else float(0), axis=1)
df = df.drop(['point','point_next'], axis = 1)

dates = df.index.floor('D')
day_distances = df["distance_km"].groupby(dates).sum()

In [21]:
day_distances

UTC_datetime
2021-01-01     3.530632
2021-01-02     1.969477
2021-01-03     4.450611
2021-01-04     0.016159
2021-01-05     0.419665
                ...    
2022-01-16    10.853490
2022-01-17    13.971461
2022-01-18    16.721647
2022-01-19    21.897270
2022-01-20     4.698010
Name: distance_km, Length: 385, dtype: float64

### Get place name from GPS

In [22]:
def get_country(latitude, longitude):
    location = geolocator.geocode(query = [latitude,longitude], addressdetails = True, language = 'en')
    address = location.raw['address']    
    country = address.get('country', '')
    return country

In [23]:
def wiki_get_place(lat,lon):
    headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64"
    }

    gs_limit = 1 # limit results
    gs_radius = 5000
    api_url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=%s|%s&gsradius=%d&gslimit=%d"
    wp_query = api_url%(lat,lon,gs_radius,gs_limit)
    print("fetching data: %s"%wp_query)

    response = requests.get(wp_query, headers=headers)
    response_data = response.json()["query"]["geosearch"]

    return response_data[0]['title']

## Create template sentences from bird data

### Location and Travel

In [24]:
def where_am_i(lat,lon):
    country = get_country(lat,lon)
    place = wiki_get_place(lat,lon)
    sentence = "I am traveling through {} near {}.".format(country, place)
    return sentence

In [25]:
def travel_summary(day):
    '''day is in the form "YYYY-MM-DD"
    '''
    
    # Total distance
    distance = round(day_distances[day])
    if distance < 11:
        sent_1 = "I didn't move around much today. I only covered about {} kilometers.".format(distance)
    if 10 < distance < 100:
        sent_1 = "I covered a fair amount of ground today. {} kilometers.".format(distance)
    if distance > 99:
        sent_1 = "I went really far today! {} kilometers!".format(distance)
    
    # Start and end point.
    day_df = df.loc[day]
    
    start_lat = day_df.iloc[0]['Latitude']
    start_lon = day_df.iloc[0]['Longitude']
    

    end_lat = day_df.iloc[-1]['Latitude']
    end_lon = day_df.iloc[-1]['Longitude']
    
    start = wiki_get_place(start_lat,start_lon)
    end = wiki_get_place(end_lat, end_lon)
    
    if start != end:
        sent_2 = 'I came here from {}.'.format(start,end)

    else:
        sent_2 = 'I returned to {} in the evening.'.format(end)
    
    # Make sentence
    sentence = " ".join([sent_1,sent_2])
    return sentence

In [26]:
travel_summary('2021-05-07')

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=45.770336|8.605195&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=45.985748|8.7455&gsradius=5000&gslimit=1


'I covered a fair amount of ground today. 61 kilometers. I came here from Taino, Lombardy.'

In [27]:
travel_summary('2021-05-05')

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.484264|11.229406&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=43.887665|10.286529&gsradius=5000&gslimit=1


'I went really far today! 193 kilometers! I came here from Albinia.'

In [28]:
travel_summary('2021-05-12')

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=46.1749|8.87764&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=46.170788|8.874135&gsradius=5000&gslimit=1


"I didn't move around much today. I only covered about 6 kilometers. I came here from Riazzino railway station."

### Weather

In [29]:
def get_weather_conditions(datetime):
    lat = df.loc[datetime]['Latitude']
    lon = df.loc[datetime]['Longitude']
    
    weather = Hourly(Point(lat,lon), datetime, datetime.ceil('H'))
    weather = weather.fetch()
    
    temp = weather['temp'][0]
    coco = weather['coco'][0]
    
    return temp, coco

In [30]:
def current_place_weather(day):
    '''Datetime should be in same format as the bird data
    e.g. '2021-01-01 06:48:35'  '''

    day_df = df.loc[day]
    datetime = day_df.index[-1]
    lat = df.loc[datetime]['Latitude']
    lon = df.loc[datetime]['Longitude']

    sent1 = where_am_i(lat,lon)
    
    temp, coco = get_weather_conditions(datetime)
    
    cond = condition(coco)

    # there's a: 17-22,
    # it's : 0-4
    # there's 5-16, >22
    if 16 < coco < 23:
        cond = "there's a" + cond
        
    elif -1 < coco < 5:
        cond = cond

    elif 4 < coco < 17 or 22 < coco:
        cond = "there's " + cond

        
    cond = cond.lower()
    
    if temp < 15:
        feel = 'a bit chilly'
    else:
        feel = 'pretty warm'
        
    sent2 = "It's {} and {} right now.".format(feel,cond) 
    
    return sent1 + ' ' + sent2

In [31]:
current_place_weather('2021-01-06')

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.480957|11.2106&gsradius=5000&gslimit=1


"I am traveling through Italy near Albinia. It's a bit chilly and fair right now."

## Topic-text from Wikipedia


### Location article

In [32]:
def get_wiki_place(datetime):
    headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64"
    }

    gs_limit = 1 # limit results
    gs_radius = 5000
    api_url = "https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=%s|%s&gsradius=%d&gslimit=%d"
    wp_query = api_url%(df.loc[datetime]["Latitude"],df.loc[datetime]["Longitude"],gs_radius,gs_limit)
    print("fetching data: %s"%wp_query)

    response = requests.get(wp_query, headers=headers)
    response_data = response.json()["query"]["geosearch"]
    for x in response_data:
        x["time"] = df.iloc[0]

    cmp_time = pd.to_datetime("2021-01-05")

    place = response_data[0]['title']
    page = wiki_wiki.page(place)
    sentences = nltk.sent_tokenize(page.text)

    return place, sentences[:5]

### Waldrapp Article

In [33]:
wiki_wiki = wikipediaapi.Wikipedia('en')

In [34]:
page = wiki_wiki.page('Northern bald ibis')

In [35]:
summary = page.summary
taxonomy = str(page.sections[0])
description = str(page.sections[1])
habitat = str(page.sections[2])
breeding_feeding = str(page.sections[3])
culture = str(page.sections[5])

feeding = breeding_feeding.split('Section: Feeding')[1]
breeding = breeding_feeding.split('Section: Feeding')[0]

## Summarize, complete etc. with GPT-3 API.

In [61]:
def GPT_Completion(texts): #, suffix = ""):
    ## Call the API key under your account (in a secure way)
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt =  texts,
        temperature = 0.7,
        top_p = 1,
        max_tokens = 300,
        frequency_penalty = 0,
        presence_penalty = 0,
        # suffix = suffix
        )
    text = response.choices[0].text
    sentences = nltk.sent_tokenize(text)
    length = len(sentences)

    if not text:
        output = ['No text generated.']

    elif sentences[-1][-1] == ".":
        output = sentences

    else:
        output = sentences[:-1]
    return " ".join(output)

# Generate Story Testing

In [ ]:
def topic_sentence(topic_text, first_person = True, past_tense = True):
    topic = GPT_Completion(topic_text + "tl;dr")
    if first_person:
        topic = GPT_Completion("Rewrite in first-person: " + topic)
    if past_tense:
        topic = GPT_Completion("Rewrite in past-tense: " + topic)
    return topic

In [ ]:
def generate_story(day, location_info = False, context = context, prompt = "", topic_text = "", style = style):
    from datetime import datetime
    date = datetime.strptime(day, '%d-%m-%Y')
    date_text = date.strftime("It is %B %d, %Y.")
    context += " " + date_text
    context += " " + current_place_weather(day)

    context_len = len(nltk.sent_tokenize(context))

    if not prompt:
        prompt = travel_summary(day)
    else:
        prompt = prompt

    if location_info:
        day_df = df.loc[day]
        datetime = day_df.index[-1]
        place, place_wiki = get_wiki_place(datetime)
        place_desc = GPT_Completion(" ".join(place_wiki) + 'tl;dr:')
        place_desc = GPT_Completion("Summarize in one sentence: " + place_desc)
        if place_desc != "No text generated.":
            prompt = place_desc + " " + prompt
        else:
            prompt = prompt

    if topic_text:
        topic = topic_sentence(topic_text)
        prompt = " ".join([context, prompt, topic])
    else:
        prompt = " ".join([context, prompt])
    
    prompt_len = len(tokenizer(prompt, return_tensors='pt').input_ids[0])
    story = " " + generate_sentences(prompt, prompt_len + 150)

    completion = GPT_Completion("Finish this story: " + story)
    if completion != 'No text generated.':
        story += " " + completion

    story = " ".join(nltk.sent_tokenize(story)[context_len:])

    if style:
        rewrite = GPT_Completion("Rewrite in {}: ".format(style) + story)

        if rewrite == 'No text generated.':
            story = story

        else:
            story = rewrite

    return print(individual + ': ' + story)

In [ ]:
generate_story('01-06-2021', location_info = True, style = style)

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.481167|11.203067&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.481167|11.203067&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.481167|11.203067&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.481167|11.203067&gsradius=5000&gslimit=1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Laura: Albinia is a town in Tuscany, central Italy. I didn't move around much today. I only covered about 1 kilometers. I flew back to Albinia in the evening. The airport was basically deserted. I was waiting for the shuttle to the train station when a guy with a dog walked in. His arms were full of a bunch of things—a bird, a cicada, a plant, and then something else. I assumed it was his carry-on—a travel bag. When the sun started to set, I asked for the gate. When I walked in, he told me that there was “no reservation” yet. He said he had only been in the airport an hour, and his flight was at 4:00 PM. The shuttle was already here waiting for him. If I took the shuttle, he said, he would wait for the train back to Florence. I told him I didn't have a reservation either. I was just waiting for the shuttle. He said I could take the shuttle with him, and he would buy my ticket. I didn't want to take his money, but he insisted. So I got on the shuttle with him and we started chatting. He

In [ ]:
prompt = "My species is important in several cultures."
topic_text = culture
# topics: summary, taxonomy,description, habitat, culture, feeding, breeding

generate_story("01-05-2021", location_info = True, prompt = prompt, topic_text = topic_text)

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.476833|11.20923&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.476833|11.20923&gsradius=5000&gslimit=1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Laura: It is now home to the Museum of the Territory. My species is important in several cultures. I was revered as a holy bird and a symbol of brilliance and splendour in Ancient Egypt, where I was regarded as an embodiment of Thoth, the scribe of the gods. After I became extinct in Central Europe, some later writers thought that Gesner's description was one of several in his book depicting mythical creatures. The Dutch naturalist Laurentius de Curiene (1585–1667) reports that some of the Iroquois regarded Iroha (the Mohawk name for the Pacific turkey vulture) as a sacred bird. In modern times, the Lakota and other northern Plains tribes regard the Pacific turkey vulture as a god, patron deity of corn. Some people seek out these birds out of a feeling that they are special and a link to a longer history. But many find that the large birds are vulnerable and vulnerable to hunters and the greed of corporations, and feel this calls their loyalty into question. I am one of the few norther

In [ ]:
def get_context(day, context = context):
    from datetime import datetime
    date = datetime.strptime(day, '%d-%m-%Y')
    date_text = date.strftime("It is %B %d, %Y.")
    context += " " + date_text
    context += " " + current_place_weather(day)
    return context

In [ ]:
get_context("01-06-2021")

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=42.481167|11.203067&gsradius=5000&gslimit=1


"Hello, my name is Laura, and I am a queer non-binary northern bald ibis. It is June 01, 2021. I am traveling through Italy near Albinia. It's a bit chilly and cloudy right now."

# Anvil WebApp Version

## Link: https://G63NADL4YNSZIG7S.anvil.app/KF5UBIFSVOWONS6LS6BZRB6P

In [37]:
!pip install anvil-uplink -qq

     |████████████████████████████████| 87 kB 3.9 MB/s 
     |████████████████████████████████| 51 kB 273 kB/s 


In [38]:
import anvil.server

In [39]:
anvil.server.connect("ANFBB6F4743XL2LALSH26SGR-G63NADL4YNSZIG7S")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment" as SERVER


In [78]:
@anvil.server.callable
def generate_story_web(day, location_info, prompt = "", topic_text = "", context = context , length = 140, style = style):
    
    # Add date, current place and weather to context
    from datetime import datetime
    date = datetime.strptime(day, '%Y-%m-%d')
    date_text = date.strftime("It is %B %d, %Y.")
    context += " " + date_text
    context += " " + current_place_weather(day)
    context_len = len(nltk.sent_tokenize(context))
    print(context)

    # Default prompt to travel summary, else use given prompt.
    if not prompt:
        prompt = travel_summary(day)
    else:
        prompt = prompt

    # Add description of location to prompt if location info set to True
    if location_info:
        day_df = df.loc[day]
        datetime = day_df.index[-1]
        place, place_wiki = get_wiki_place(datetime)
        place_wiki = " ".join(place_wiki)
        place_desc = GPT_Completion(place_wiki + 'tl;dr:')
        # place_desc = GPT_Completion("Summarize in one sentence: " + place_wiki)
        if place_desc != "No text generated.":
            prompt = " ".join([place_desc,prompt])
        else:
            prompt = prompt

    # Add topic sentence to prompt, if specified.
    # Generate initial part of story.
    if topic_text:
        if topic_text != "No text generated.":
            topic = topic_text
            prompt = " ".join([context, prompt, topic])
    else:
        prompt = " ".join([context, prompt])

    prompt_len = len(tokenizer(prompt, return_tensors='pt').input_ids[0])
    story = " " + generate_sentences(prompt, prompt_len + length)

    # Use GPT-3 to complete the story
    completion = GPT_Completion("Finish this story: " + story)
    if completion != 'No text generated.':
        story += " " + completion

    story = " ".join(nltk.sent_tokenize(story)[context_len:])

    # If style is specified, rewrite everything in given style.
    if style:
        rewrite = GPT_Completion("Rewrite in {}: ".format(style) + story)

        if rewrite == 'No text generated.':
            story = story

        else:
            story = rewrite

    # Insert sentence about current location to the story beginning if location
    # info is desired.
    if location_info:
        place_text = nltk.sent_tokenize(context)[-2]
        story = place_text + " " + story

    return story

In [75]:
@anvil.server.callable
def topic_sentence_web(topic_text, first_person = True, past_tense = True):
    topic = GPT_Completion(topic_text + "tl;dr")
    if first_person:
        topic = GPT_Completion("Rewrite in first-person: " + topic)
    if past_tense:
        topic = GPT_Completion("Rewrite in past-tense: " + topic)
    return topic

In [76]:
@anvil.server.callable
def get_context(day, context = context, style = style):
    from datetime import datetime
    date = datetime.strptime(day, '%Y-%m-%d')
    date_text = date.strftime("It is %B %d, %Y.")
    context += " " + date_text
    context += " " + current_place_weather(day)
    if style:
        rewrite = GPT_Completion("Rewrite in {}: ".format(style) + context)

        if rewrite == 'No text generated.':
            context = context

        else:
            context = rewrite
    return context

In [77]:
anvil.server.wait_forever()

fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
Hello, my name is Laura, and I am a queer non-binary northern bald ibis. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now.
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.216316|11.566822&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
Hello, my name is Laura, and I am a queer non-binary northern bald ibis. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now.
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.216316|11.566822&gsradius=5000&gslimit=1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
Hello, my name is Laura, and I am a queer non-binary northern bald ibis. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now. It is May 10, 2021. I am traveling through Germany near Harthof. It's pretty warm and cloudy right now.
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.216316|11.566822&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=48.211334|11.571905&gsradius=5000&gslimit=1


...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=47.764698|12.442179&gsradius=5000&gslimit=1
Hello, my name is Laura, and I am a queer non-binary northern bald ibis. It is May 20, 2021. I am traveling through Germany near Marquartstein. It's a bit chilly and overcast right now. It is May 20, 2021. I am traveling through Germany near Marquartstein. It's a bit chilly and overcast right now.
fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=47.87125|12.583041&gsradius=5000&gslimit=1


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


fetching data: https://en.wikipedia.org/w/api.php?action=query&format=json&list=geosearch&gscoord=47.764698|12.442179&gsradius=5000&gslimit=1


KeyboardInterrupt: ignored